<a href="https://colab.research.google.com/github/zhuoyingcai/NYC-Housing-Prices-Analysis/blob/master/NYC_Housing_Prices_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Section

In [0]:
import os

# Data
import numpy as np
import pandas as pd

# Plot
import matplotlib.pyplot as plt


# Path Functions

In [0]:
def ensure_dir(path_to_dir):
  try:
      os.makedirs(path_to_dir)
  except FileExistsError:
      # directory already exists
      pass

# Config

In [3]:
# path config
annualizedSalesUpdate_data_dir = '/content/drive/My Drive/Colab Notebooks/dataScience/annualizedSalesUpdate'
data_dir = '/content/drive/My Drive/Colab Notebooks/dataScience/data'

# mount google drive
from google.colab import drive
drive.mount('/content/drive')

# make sure dir exist
ensure_dir(data_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load and summarize data

In [0]:
# define dataframe
df = pd.DataFrame()

In [0]:
years = [2017, 2018]
boroughs = ['bronx', 'brooklyn', 'manhattan', 'queens', 'statenisland']

# load data
for year in years:
  for borough in boroughs:
    path = annualizedSalesUpdate_data_dir + '/%s_%s.xls' % (year, borough)
    data = pd.read_excel(path, header=4)
    df = df.append(data, sort=False)

We first visually inspect the data to see if there is any missing data or incorrect data types. Also, we may want to edit data entry.

In [6]:
# df_bronx.head(20)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170407 entries, 0 to 8385
Data columns (total 21 columns):
BOROUGH                           170407 non-null int64
NEIGHBORHOOD                      170407 non-null object
BUILDING CLASS CATEGORY           170407 non-null object
TAX CLASS AT PRESENT              170349 non-null object
BLOCK                             170407 non-null int64
LOT                               170407 non-null int64
EASE-MENT                         88660 non-null object
BUILDING CLASS AT PRESENT         170349 non-null object
ADDRESS                           170407 non-null object
APARTMENT NUMBER                  106536 non-null object
ZIP CODE                          170393 non-null float64
RESIDENTIAL UNITS                 170349 non-null float64
COMMERCIAL UNITS                  170349 non-null float64
TOTAL UNITS                       170349 non-null float64
LAND SQUARE FEET                  170346 non-null float64
GROSS SQUARE FEET                 1

In [7]:
df.head(10)

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3028,25,,A5,412 EAST 179 STREET,,...,1.0,0.0,1.0,1842.0,2048.0,1901.0,1,A5,0,2017-04-04
1,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3030,55,,A5,410 EAST 182ND STREET,,...,1.0,0.0,1.0,1330.0,1460.0,1899.0,1,A5,305000,2017-07-18
2,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3030,56,,A1,412 EAST 182 STREET,,...,1.0,0.0,1.0,1306.0,1440.0,1899.0,1,A1,178000,2017-01-19
3,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3030,56,,A1,412 EAST 182 STREET,,...,1.0,0.0,1.0,1306.0,1440.0,1899.0,1,A1,449000,2017-07-14
4,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3030,65,,A1,4455 PARK AVENUE,,...,1.0,0.0,1.0,1622.0,1587.0,1899.0,1,A1,140000,2017-05-12
5,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3030,70,,A1,4445 PARK AVENUE,,...,1.0,0.0,1.0,1694.0,1497.0,1899.0,1,A1,246000,2017-11-06
6,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3036,13,,A1,4348 PARK AVENUE,,...,1.0,0.0,1.0,3525.0,1764.0,1899.0,1,A1,420000,2017-02-03
7,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3037,42,,A1,4428 PARK AVENUE,,...,1.0,0.0,1.0,3525.0,1340.0,1899.0,1,A1,380250,2017-09-29
8,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3037,101,,S0,443 EAST 180 STREET,,...,1.0,2.0,3.0,1293.0,2820.0,1952.0,1,S0,325000,2017-11-13
9,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3043,55,,A1,1948 BATHGATE AVENUE,,...,1.0,0.0,1.0,2356.0,2047.0,1901.0,1,A1,127000,2017-08-22


In [8]:
df.tail()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
8381,5,WOODROW,05 TAX CLASS 1 VACANT LAND,1B,7094,61,NaN,V0,23 WILTSHIRE LANE,NaN,...,0.0,0.0,0.0,4427.0,0.0,2018.0,1,V0,793216,2018-05-25
8382,5,WOODROW,05 TAX CLASS 1 VACANT LAND,1,7267,289,NaN,A5,41 JADE COURT,NaN,...,1.0,0.0,1.0,2955.0,2612.0,2018.0,1,V0,671026,2018-10-04
8383,5,WOODROW,05 TAX CLASS 1 VACANT LAND,1,7267,298,NaN,A5,11 JADE COURT,NaN,...,1.0,0.0,1.0,4108.0,2612.0,2018.0,1,V0,610950,2018-07-23
8384,5,WOODROW,05 TAX CLASS 1 VACANT LAND,1,7267,299,NaN,A5,9 JADE COURT,NaN,...,1.0,0.0,1.0,11019.0,2612.0,2018.0,1,V0,672043,2018-08-07
8385,5,WOODROW,05 TAX CLASS 1 VACANT LAND,1B,7346,1,NaN,V0,GAYNOR STREET,NaN,...,0.0,0.0,0.0,42016.0,0.0,0.0,1,V0,0,2018-09-21


We found two problems:

 1. we see that the dataframe has 170407 entries, but TAX CLASS AT PRESENT, EASE-MENT, BUILDING CLASS AT PRESENT, APARTMENT NUMBER, ZIP CODE, RESIDENTIAL UNITS, COMMERCIAL UNITS, TOTAL UNITS, LAND SQUARE FEET, GROSS SQUARE FEET and YEAR BUILT has less entries.
 
 2. The BROUGH is represented as numerical values rather than categorical.
 
 3. There are some data does not make sense, such as GROSS SQUARE FEET, YEAR BUILT and SALE PRICE can not be 0.


---


 What we are going to do next:
 
 1. rename BOROUGH values to actual borough name.
 
 2. drop columns that we are not going to use.
 
 3. remove raws with 0 SALE PRICE, GROSS SQUARE FEET, YEAR BUILT.

In [0]:
df['BOROUGH'] = df['BOROUGH'].replace({1: 'Manhattan', 2: 'Bronx', 3: 'Brooklyn', 4: 'Queens', 5: 'Staten Island'})

In [0]:
df = df.drop(['BUILDING CLASS CATEGORY', 'TAX CLASS AT PRESENT', 'EASE-MENT', 'BUILDING CLASS AT PRESENT', 'ADDRESS',
         'APARTMENT NUMBER', 'ZIP CODE', 'BUILDING CLASS AT TIME OF SALE' ], axis=1)

Empty DataFrame
Columns: [SALE PRICE]
Index: []
